# Cache

- Caches can help you save money, simply be storing results from frequent queries / computationally heavy queries where it is easily looked up!
    - i.e. check if the question has been asked before, if it has, just return whatever you answered previously

- DBs can deal with uneven loads of reads/writes by time. With a cache, you prevent spikes in DB hits

## Types of Caching

- Client caching
    - Store a copy of the response on the client side (i.e. in the user's browser/device)

- CDN caching
    - CDNs can store a copy of the result for an entire segment of users

- Web server caching
    - Reverse proxies can cache results of frequent queries

- Database caching
    - DB implements caching to avoid computational hits

- Application caching
    - In-memory caches like Redis/Memcached can sit between your application and DB
    - Data is stored in RAM, so it is fast
    - But RAM is limited
    - So you need a decent cache invalidation strategy. Typical strategy is to drop least recently used (LRU) entries

## Granularity of Caching

- Query Level
    - When a query hits your DB, hash the query and cache the result
    - BUT downsides are:
        - Hard to delete a cached result with complex queries, unless you preset an expiration time
        - If one piece of data changes such as a table cell, you need to delete all cached queries that might include the changed cell

- Object level
    - Your query result is an object
    - Hash it and cache it

## Updating Cache

- Space is always limited, so you can't cache everything. 
- So you need to choose a cache update strategy from below

- Cache-aside
    - What is it?
        - Application reads and writes from storage, and is in charge of caching. Cache does not interact with DB
        - Steps:
            - Application will look for entry in cache.
            - If cache miss, look up DB
            - Add entry to cache
            - Return
    - Disadvanges
        - Each cache miss takes three trips (get result from DB, compute, and add to cache), which can cause a noticeable delay.
        - Cached data can become stale if DB is updated. Mitigate by setting a time-to-live (TTL) 
        - When a node fails, it is replaced by a new, empty node, so everything has to be recached.

- Write-through
    - What is it?
        - Cache is used as the main DB, so you always read from and write to the cache. The cache then updates the DB
        - This way, always guarantee that cache data is not stale
        - Users generally are more ok with latency when writing data than when reading
    - Disadvanges
        - This is obviously slow because you need to write to 2 places, but subsequent reads are fast. 
        - If a node fails, the new node will not have cache entries until the entry is updated in the DB. 
        - Wasted space in cache because most data written might never be read

- Write-behind (write-back)
    - What is it?
        - Add/update entry in cache
        - Asynchronously write entry to the data store, improving write performance because you don't need to wait until the DB write finishes before returning to user
    - Disadvanges
        - If you cache dies before the async write is complete, you can lose data
        - More complex to implement write-behind than it is to implement cache-aside or write-through.

- Refresh-ahead
    - What is it?
        - Cache automatically refreshes any recently accessed cache entry prior to its expiration.
        - Can result in reduced latency vs read-through if the recently accessed cache entries are predictive of future needs 

    - Disadvanges
        - If past use is not predictive of future use, then you may be storing a lot of junk in cache for nothing

## General disadvantages of caching

-  Need to maintain consistency between caches and the source of truth such as the database through cache invalidation.
- Cache invalidation is a difficult problem, there is additional complexity associated with when to update the cache.
- Need to make application changes such as adding Redis or memcached.